In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import tensorflow as tf
print(tf.__version__)

Using TensorFlow backend.
c:\users\burak\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\burak\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\burak\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

1.13.1


In [2]:
df = pd.read_csv('D:/DeepLearning/Text-Classification-Models/Data/train.csv')

In [3]:
df.head()

,author,text
0,EAP,"This process, however, afforded me no means of..."
1,HPL,It never once occurred to me that the fumbling...
2,EAP,"In his left hand was a gold snuff box, from wh..."
3,MWS,How lovely is spring As we looked from Windsor...
4,HPL,"Finding nothing else, not even gold, the Super..."


In [5]:
df.author.unique()

array(['EAP', 'HPL', 'MWS'], dtype=object)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19579 entries, 0 to 19578
Data columns (total 2 columns):
author    19579 non-null object
text      19579 non-null object
dtypes: object(2)
memory usage: 306.0+ KB


In [7]:
df['text']=df['text'].str.lower()

In [8]:
def getLemmText(text):
    tokens=word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)
df['text'] = list(map(getLemmText,df['text']))

In [9]:
def getStemmText(text):
    tokens=word_tokenize(text)
    ps = PorterStemmer()
    tokens=[ps.stem(word) for word in tokens]
    return ' '.join(tokens)
df['text'] = list(map(getStemmText,df['text']))

In [10]:
xtrain, xtest, ytrain, ytest = train_test_split(
 df['text'], df['author'], 
 test_size=0.33, 
 random_state=53)
print(xtrain.shape)
print(xtest.shape)
print(ytrain)

(13117,)
(6462,)
6417     MWS
1105     MWS
12486    EAP
18197    EAP
11767    MWS
3083     HPL
18708    MWS
363      HPL
1834     EAP
9430     EAP
3333     EAP
12083    EAP
11220    EAP
10994    EAP
9497     HPL
2521     EAP
15693    EAP
18184    MWS
3766     EAP
11602    EAP
12878    EAP
1875     MWS
7325     EAP
14449    EAP
9677     EAP
10188    HPL
17630    EAP
14790    EAP
14211    MWS
18499    MWS
        ... 
2747     MWS
8035     MWS
8924     HPL
9488     MWS
234      EAP
8400     EAP
11790    HPL
4102     HPL
11787    EAP
2339     EAP
13310    EAP
3226     EAP
8237     MWS
15625    EAP
17947    MWS
19159    HPL
18778    EAP
13606    MWS
13268    EAP
11211    EAP
13401    EAP
3269     HPL
2441     EAP
1164     MWS
11450    HPL
8024     EAP
4596     HPL
8854     EAP
14075    HPL
2933     EAP
Name: author, Length: 13117, dtype: object


In [11]:
EMBEDDING_DIMENSION = 64
VOCABULARY_SIZE = 2000
MAX_LENGTH = 100
OOV_TOK = '<OOV>'
TRUNCATE_TYPE = 'post'
PADDING_TYPE = 'post'

In [12]:
tokenizer = Tokenizer(num_words=VOCABULARY_SIZE, oov_token=OOV_TOK)
tokenizer.fit_on_texts(list(xtrain) + list(xtest))

In [13]:
xtrain_sequences = tokenizer.texts_to_sequences(xtrain)
xtest_sequences = tokenizer.texts_to_sequences(xtest)
word_index = tokenizer.word_index
print('Vocabulary size:', len(word_index))
dict(list(word_index.items())[0:10])
with open('personal.json', 'w') as json_file:
    json.dump(word_index, json_file)

Vocabulary size: 15762


In [14]:
print(xtrain_sequences[100])

[27, 6, 113, 2, 424, 94, 29, 1, 40, 37, 110, 1254, 4, 9, 103, 6, 599, 40, 1027, 122, 23, 805, 24, 38, 3, 2, 1, 1, 1, 564, 1, 6, 114, 1755, 1, 19, 8, 528, 4, 1, 57, 1, 920, 69, 8, 54, 1, 117, 44, 2, 826, 3, 744, 1]


In [15]:
xtrain_pad = sequence.pad_sequences(xtrain_sequences, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)
xtest_pad = sequence.pad_sequences(xtest_sequences, maxlen=MAX_LENGTH, padding=PADDING_TYPE, truncating=TRUNCATE_TYPE)
print(len(xtrain_sequences[0]))
print(len(xtrain_pad[0]))
print(xtrain_pad[100])

13
100
[  27    6  113    2  424   94   29    1   40   37  110 1254    4    9
  103    6  599   40 1027  122   23  805   24   38    3    2    1    1
    1  564    1    6  114 1755    1   19    8  528    4    1   57    1
  920   69    8   54    1  117   44    2  826    3  744    1    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]


In [16]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(list(ytrain))
training_label_seq = np.array(label_tokenizer.texts_to_sequences(ytrain))
test_label_seq = np.array(label_tokenizer.texts_to_sequences(ytest))
print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

[2]
[2]
[1]
(13117, 1)


In [17]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
def decode_article(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
print(decode_article(xtrain_pad[11]))

weari at heart with anxieti which had their origin in the gener <OOV> and decay i <OOV> to the <OOV> fever ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?


In [18]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     EMBEDDING_DIMENSION))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(EMBEDDING_DIMENSION, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(EMBEDDING_DIMENSION, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(EMBEDDING_DIMENSION, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(4))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          1008832   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, None, 64)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)      

In [22]:
num_epochs = 10
history = model.fit(xtrain_pad, training_label_seq, epochs=num_epochs, validation_data=(xtest_pad, test_label_seq), verbose=2)

Train on 13117 samples, validate on 6462 samples
Epoch 1/10
 - 43s - loss: 0.6795 - acc: 0.7509 - val_loss: 0.5855 - val_acc: 0.7629
Epoch 2/10
 - 43s - loss: 0.6265 - acc: 0.7765 - val_loss: 0.5715 - val_acc: 0.7751
Epoch 3/10
 - 45s - loss: 0.6033 - acc: 0.7843 - val_loss: 0.5842 - val_acc: 0.7745
Epoch 4/10
 - 44s - loss: 0.5801 - acc: 0.7974 - val_loss: 0.5734 - val_acc: 0.7767
Epoch 5/10
 - 40s - loss: 0.5534 - acc: 0.8085 - val_loss: 0.6029 - val_acc: 0.7795
Epoch 6/10
 - 40s - loss: 0.5301 - acc: 0.8157 - val_loss: 0.6111 - val_acc: 0.7731
Epoch 7/10
 - 40s - loss: 0.5159 - acc: 0.8228 - val_loss: 0.6022 - val_acc: 0.7744
Epoch 8/10
 - 40s - loss: 0.5099 - acc: 0.8214 - val_loss: 0.6569 - val_acc: 0.7640
Epoch 9/10
 - 41s - loss: 0.4870 - acc: 0.8301 - val_loss: 0.6689 - val_acc: 0.7620
Epoch 10/10
 - 41s - loss: 0.4790 - acc: 0.8363 - val_loss: 0.6896 - val_acc: 0.7724
